#### ST

#### MST
- kruskal algorithm
- prim's algorithm

#### kruskal
- 시간복잡도 비교
- 논리 정리 (응용위한)

In [2]:
mygraph = {
    'vertices': ['A', 'B', 'C', 'D', 'E', 'F', 'G'],
    'edges': [
        (7, 'A', 'B'),
        (5, 'A', 'D'),
        (7, 'B', 'A'),
        (8, 'B', 'C'),
        (9, 'B', 'D'),
        (7, 'B', 'E'),
        (8, 'C', 'B'),
        (5, 'C', 'E'),
        (5, 'D', 'A'),
        (9, 'D', 'B'),
        (7, 'D', 'E'),
        (6, 'D', 'F'),
        (7, 'E', 'B'),
        (5, 'E', 'C'),
        (7, 'E', 'D'),
        (8, 'E', 'F'),
        (9, 'E', 'G'),
        (6, 'F', 'D'),
        (8, 'F', 'E'),
        (11, 'F', 'G'),
        (9, 'G', 'E'),
        (11, 'G', 'F')
    ]
}

parent = dict()
rank = dict()

def make_set(node):
    parent[node] = node  # root
    rank[node] = 0
    
def find(node):
    if parent[node] != node:
        parent[node] = find(parent[node])
    return parent[node]

def union(nodeA, nodeB):
    rootA = find(nodeA)
    rootB = find(nodeB)
    
    if rootA > rootB:
        parent[rootB] = rootA
    else:
        parent[rootA] = rootB
        if rank[rootA] == rank[rootB]:
            rank[rootB] += 1

def kruskal(start, graph):
    # 모든 간선들 오름차순 순으로 놓고서 트리 형태로 붇도록
    mst = []
    for node in graph['vertices']:  # 수정1 :  노드로만   # O(V)
        make_set(node)
        
    edges = graph['edges']
    edges.sort()      # O ( ElogE )  (퀵소트 : O ( NlogN))
    
    for weight, nodeA, nodeB in edges:
        if find(nodeA) != find(nodeB):   # union - find : O(logN) -> O(E) ; O(1)
            union(nodeA, nodeB)
            mst.append((weight,nodeA, nodeB))
    return mst

kruskal('A', mygraph)

[(5, 'A', 'D'),
 (5, 'C', 'E'),
 (6, 'D', 'F'),
 (7, 'A', 'B'),
 (7, 'B', 'E'),
 (9, 'E', 'G')]

#### prim's 
- 시간복잡도가 kruskal보다 더 오래 걸림
- 개선된 프림 알고리즘의 시간복잡도 , 데이터 형태, 각각 필요한 라이브러리

In [9]:
myedges = [
    (7, 'A', 'B'), (5, 'A', 'D'),
    (8, 'B', 'C'), (9, 'B', 'D'), (7, 'B', 'E'),
    (5, 'C', 'E'),
    (7, 'D', 'E'), (6, 'D', 'F'),
    (8, 'E', 'F'), (9, 'E', 'G'),
    (11, 'F', 'G')
]

from heapq import *
from collections import defaultdict
def before_prim(start, edges):
    #  데이터는 그냥 weight, node->node 순 
    #  간선들 중심, heapify, heappop( list ),  defaultdict
    mst = []
    adjacent_edges = defaultdict(list)     # 수정1 : 안에다 어떤 자료형 쓸 건지 적어줘야 해
    for weight, nodeA, nodeB in edges:
        adjacent_edges[nodeA].append((weight, nodeA, nodeB))   # 마지막, 두번째 노드는 다음노드로
        adjacent_edges[nodeB].append((weight, nodeB, nodeA))
    
    connected_edges = set(start)
    candidate_edges = adjacent_edges[start]
    heapify(candidate_edges)    # 최소 힙 구조!   => O ( ElogE )
    
    while candidate_edges:   # 최악의 경우 모든 간선 반복
        weight, nodeA, nodeB = heappop(candidate_edges)
        if nodeB not in connected_edges:
            connected_edges.add(nodeB)
            mst.append((weight, nodeA, nodeB))
            
            for edge in adjacent_edges[nodeB]:
                if edge[2] not in connected_edges:
                    candidate_edges.append(edge)
    return mst


from heapdict import heapdict

def prim(start, graph):
    # edge보다 node의 개수가 작은 걸 이용, heapdict -> popitem()
    # 데이터는 node : weight
    # keys와 pi 로 돌아가는 것만 이해하면 간단
    mst, keys, pi, total_weight = [], heapdict(), dict(), 0
    
    for node in graph.keys():    # 노드 중심!
        keys[node] = float('inf')    # 우선순위 큐   #  O(V)
        pi[node] = None
    keys[start], pi[start] = 0, start
    
    while keys:   # 노드 개수만큼 V
        # 바로 연결 -> 즉 다음 노드 연결시 선별과정!
        current_node, current_weight = keys.popitem()    # extract min logic - O(logV)  -> while과 같이 보면 O(VlogV)
        mst.append((current_node, current_weight))
        total_weight += current_weight
        
        for adjacent, weight in graph[current_node].items():    # 데이터 키에 키와 벨류들 있다!   # BigO : 최대 간선 수 만큼 : O(E)
            if adjacent in keys and keys[adjacent] > weight:
                keys[adjacent] = weight    #  값 갱신시 우선순위 큐 재구성 :"decrease key" - heapdict 라이브러리로  # heap 구조변경 : O(logV)
                pi[adjacent] = current_node
                
    return mst, total_weight

mygraph = {
    'A': {'B': 7, 'D': 5},
    'B': {'A': 7, 'D': 9, 'C': 8, 'E': 7},
    'C': {'B': 8, 'E': 5},
    'D': {'A': 5, 'B': 9, 'E': 7, 'F': 6},
    'E': {'B': 7, 'C': 5, 'D': 7, 'F': 8, 'G': 9},
    'F': {'D': 6, 'E': 8, 'G': 11},
    'G': {'E': 9, 'F': 11}    
}

print(before_prim('A', myedges))

print(prim('A', mygraph))
#  총 O(v + VlogV + ElogV ) => O( ElogV) 

[(5, 'A', 'D'), (7, 'A', 'B'), (6, 'D', 'F'), (7, 'B', 'E'), (5, 'E', 'C'), (9, 'E', 'G')]
([('A', 0), ('D', 5), ('F', 6), ('B', 7), ('E', 7), ('C', 5), ('G', 9)], 39)
